In [2]:
import pandas as pd
import yfinance as yf
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
from psycopg2.errors import UniqueViolation
from fin_func import *

In [3]:
# Database connection details
database = 'fin_data'
user = 'stevengiallourakis'
password = 'Birdman11!' # this might not be needed depenending on where you are connecting from.
host = 'localhost'
port =  '5433'
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [ ]:
# DB = stocks, table = stocks

stock_data = get_stock_data(symbol, start_date, end_date)

# add data to database - table stocks

data_to_db(stock_data, 'stocks', engine)

print(stock_data)

In [ ]:
# Details for commodities api request

symbol = ''
start_date = '2020-01-01'
end_date = '2024-07-04'

In [ ]:
# DB = stocks, table = commodities
commodities_data = get_commodities_data(symbol, start_date, end_date)

data_to_db(commodities_data, 'commodities', engine)

print(commodities_data)

In [ ]:
# Bulk Data download

bulk_data = []

In [ ]:
# Bulk Data download



bulk_data = []

# Iterate through list of stocks

for symbol in bulk_data:
        print("Retrieving stock data")
            
        # Retrieve stock data and update dataframe to fit database
        stock_data = get_stock_data(symbol, start_date, end_date)


        # Check if the stock data is successfully downloaded
        if stock_data.empty:
            print(f"Failed to download data for {symbol}")
        else:
            print(f"Stock data downloaded and updated for {symbol}")

    
        # Send data to database
        data_to_db(daily_stock_prices, 'stocks', engine)

In [4]:
# This code is designed to create a list of companies by letter.
# This is being done so that I can break up the size of the data downloads from yfinance.
# I can designate the letter i want to start with and the code will create a list for companies
# that start with that letter.

def create_company_list_by_letter(letter, engine):

    # Connect to the database
    with engine.connect() as connection:
        # Query to select companies where the name starts with the specified letter
        result = connection.execute(
            text("SELECT stock_symbol, company_name FROM company_list WHERE company_name LIKE :letter"),
            {"letter": f"{letter}%"}
        ).fetchall()

    # Convert result to a list of dictionaries
    companies = [{"stock_symbol": row[0], "company_name": row[1]} for row in result]
    return companies



In [5]:
letter = 'A'
# Letters downloaded and added to database
companies = create_company_list_by_letter(letter, engine)
print(f"Companies starting with {letter}:")
for company in companies:
    print(company)

Companies starting with A:
{'stock_symbol': 'A', 'company_name': 'Agilent Technologies'}
{'stock_symbol': 'AA', 'company_name': 'Alcoa Inc.'}
{'stock_symbol': 'AAN', 'company_name': "Aaron's Inc"}
{'stock_symbol': 'AAT', 'company_name': 'American Assets Trust'}
{'stock_symbol': 'AAV', 'company_name': 'Advantage Oil & Gas Ltd'}
{'stock_symbol': 'AB', 'company_name': 'Alliance Capital Management L.P.'}
{'stock_symbol': 'ABB', 'company_name': 'Abb Ltd'}
{'stock_symbol': 'ABBV', 'company_name': 'Abbvie Inc. Common Stock'}
{'stock_symbol': 'ABC', 'company_name': 'Amerisourcebergen Corporation'}
{'stock_symbol': 'ABM', 'company_name': 'Abm Industries Incorporated'}
{'stock_symbol': 'ABR', 'company_name': 'Arbor Realty Trust'}
{'stock_symbol': 'ABR-PA', 'company_name': 'Arbor Realty Trust Preferred Se'}
{'stock_symbol': 'ABR-PB', 'company_name': 'Arbor Realty Trust Cumulative R'}
{'stock_symbol': 'ABT', 'company_name': 'Abbot Laboratories'}
{'stock_symbol': 'ABW-PB', 'company_name': 'Associat

In [12]:
def get_stock_data(stock_symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    if not isinstance(stock_symbol, str):
        raise TypeError("'stock_symbol' must be a string representing a valid stock symbol")
    try:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    except Exception as e:
        logging.error(f"Error converting dates to datetime: {e}")
        return None
    try:
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    except Exception as e:
        logging.error(f"Error downloading stock data: {e}")
        return None

    if stock_data.empty:
        logging.error(f"Error stock data received. Aborting further operations.")
        return None

    # Drop unnecessary columns
    stock_data = stock_data.drop(columns=['High', 'Low', 'Adj Close', 'Volume'], errors='ignore')

    # Calculate daily change and percent change
    stock_data['daily_change'] = stock_data['Close'] - stock_data['Open']
    stock_data['percent_change'] = (
            ((stock_data['Close'] - stock_data['Open']) / stock_data['Open']) * 100)

    # Add the stock symbol column
    stock_data['stock_symbol'] = stock_symbol

    
    stock_data.reset_index(inplace=True)
    logging.info("Stock_data downloaded, High and Low columns removed, daily_change added")
    return stock_data



def data_to_db(df, table_name, engine):
    try:
        with engine.begin() as connection:
            for index, row in df.iterrows():
                sql = text(f"""
                    INSERT INTO {table_name} ("Date", "Open", "Close", daily_change, percent_change, stock_symbol)
                    VALUES (:Date, :Open, :Close, :daily_change, :percent_change, :stock_symbol)
                    ON CONFLICT (stock_symbol, "Date") DO NOTHING
                """)
                connection.execute(sql, {
                    'Date': row['Date'],
                    'Open': row['Open'],
                    'Close': row['Close'],
                    'daily_change': row['daily_change'],
                    'percent_change': row['percent_change'],
                    'stock_symbol': row['stock_symbol']
                })
        

        
        #df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        #print("Data successfully added to the database.")
    
    except UniqueViolation as e:
        print(f"Duplicate entry found for {stock_symbol} on {data['date']}. Skipping insert.")
    except Exception as e:
        print(f"An error occurred while inserting data: {e}")
    

In [13]:
for company in companies:
    stock_symbol = company["stock_symbol"]
    start_date = '2010-01-01'
    end_date = '2024-07-01'
    stock_data = get_stock_data(stock_symbol, start_date, end_date)
    if stock_data is not None:
        data_to_db(stock_data, 'daily_stock_prices', engine)
    else:
        print(f"Failed to retrieve data for {stock_symbol}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAV']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2010-01-01 00:00:00 -> 2024-07-01 00:00:00)')
ERROR:root:Error stock data received. Aborting further operations.


$AAV: possibly delisted; No price data found  (1d 2010-01-01 00:00:00 -> 2024-07-01 00:00:00)
Failed to retrieve data for AAV


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ABB']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.


Failed to retrieve data for ABB


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ABC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['EPAX']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2010-01-01 00:00:00 -> 2024-07-01 00:00:00)')
ERROR:root:Error stock data received. Aborting further operations.


$EPAX: possibly delisted; No price data found  (1d 2010-01-01 00:00:00 -> 2024-07-01 00:00:00)
Failed to retrieve data for EPAX


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PETX']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.


Failed to retrieve data for PETX


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
print(stock_data)